In [1]:
#Using Pandas as the data processing package.
import pandas as pd

#Using plotly as the visualization package.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, dcc, html, Input, Output,callback

#for formatting, we will import the dash bootstrap components.
import dash_bootstrap_components as dbc

In [2]:
app = Dash(
    __name__, 
    external_stylesheets=[dbc.themes.LUMEN]
)

Creating function for Tab Creation

In [3]:
def tab_name(tab_contents,label):
    return dbc.Tab(
        tab_contents, 
        label = label, 
        activeLabelClassName="alert-warning",
    )  

Creating function for cardHeader

In [4]:
def card(name):
    return dbc.CardHeader([
       html.Div([
                    html.H2(
                        name, 
                        className="mt-4 px-4 align-center"), 
                ]),
            ])

Creating fucntion for DropDown below Header 

In [5]:
def dropdown(label,options):
    return dbc.Col([
                html.Div([
                    dbc.Label(label, className="pl-4"),
                    options,
                ], className="col-6")
            ], className="my-4")

Creating function for Dropdown

In [6]:
def dcc_dropdown(options,first_value,id):
    return dcc.Dropdown(
        
    options=options,
    value=first_value,
    id = id,
    clearable=False
)

Creating Function for Graph

In [7]:
def tab_graph(graphheading,graph):
    return dbc.Col([
                html.Div([
                    html.H5(graphheading, className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                graph
    ])

In [8]:
unempl = pd.read_csv('Output_rates_file.csv')
unempl.head(10)

,observation_date,Labor_Rate_Names,Labor_Rate_Values,State,DISCWORK,ALLMARGAT,CIVLF,EMPLOY,INVOLPTEMP,UNEMPLOY,UNEMP15,JOBLOS
0,01-01-2004,ALLMARGAT_Rate,1.323962,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300
1,01-01-2004,EMPLOY_Rate,93.529893,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300
2,01-01-2004,INVOLPTEMP_Rate,2.766836,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300
3,01-01-2004,UNEMP15_Rate,2.997229,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300
4,01-01-2004,UNEMPLOY_Rate,6.470107,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300
5,01-01-2004,JOBLOS_Rate,3.666111,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300
6,01-04-2004,ALLMARGAT_Rate,1.432506,NewYork,46400,134200,9368200,8768400,248200,599900,275100,334300
7,01-04-2004,EMPLOY_Rate,93.597489,NewYork,46400,134200,9368200,8768400,248200,599900,275100,334300
8,01-04-2004,INVOLPTEMP_Rate,2.649388,NewYork,46400,134200,9368200,8768400,248200,599900,275100,334300
9,01-04-2004,UNEMP15_Rate,2.936530,NewYork,46400,134200,9368200,8768400,248200,599900,275100,334300


In [9]:
unempl['Labor_Rate_Names'].unique()

# ALLMARGAT_Rate : All Marginally Attached Workers
# INVOLPTEMP_Rate : Employed Involuntary Part-Time
# UNEMP15_Rate: Civilians Unemployed for min 15 Weeks
# JOBLOS_Rate_Rate : Job losers
# UNEMPLOY_Rate : Total Unemployment Workers
dic_names = {
    'ALLMARGAT_Rate' : 'All Marginally Attached Workers',
    'INVOLPTEMP_Rate' : 'Employed Involuntary Part-Time',
    'UNEMP15_Rate': 'Civilians Unemployed for min 15 Weeks',
    'JOBLOS_Rate' : 'Job losers',
    'UNEMPLOY_Rate' : 'Total Unemployment Workers',
    'EMPLOY_Rate': 'All Employed Workers'
}
def group(option):
    if option in ['All Marginally Attached Workers', 'Employed Involuntary Part-Time']:
        v = 'Unemployemnt Level 1'
    elif option in ['Civilians Unemployed for min 15 Weeks','Job losers']:
          v = 'Unemployemnt Level 2'
    else:
        v = option
    return v
    
unempl['Labor_Rate_Names'] = unempl['Labor_Rate_Names'].replace(dic_names)
unempl['unemployement_levels']=unempl['Labor_Rate_Names'].apply(group)
unempl.head()


,observation_date,Labor_Rate_Names,Labor_Rate_Values,State,DISCWORK,ALLMARGAT,CIVLF,EMPLOY,INVOLPTEMP,UNEMPLOY,UNEMP15,JOBLOS,unemployement_levels
0,01-01-2004,All Marginally Attached Workers,1.323962,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300,Unemployemnt Level 1
1,01-01-2004,All Employed Workers,93.529893,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300,All Employed Workers
2,01-01-2004,Employed Involuntary Part-Time,2.766836,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300,Unemployemnt Level 1
3,01-01-2004,Civilians Unemployed for min 15 Weeks,2.997229,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300,Unemployemnt Level 2
4,01-01-2004,Total Unemployment Workers,6.470107,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300,Total Unemployment Workers


In [10]:
unempl['observation_date'] = pd.to_datetime(unempl['observation_date'],format = '%d-%m-%Y')
unempl = unempl.sort_values('observation_date')
unempl_recess = unempl[unempl['observation_date'] <= '2014']
def indicator(num):
    if num.year <= 2007:
        return "Before Recession"
 
    elif num.year >= 2008 and num.year < 2012:
        return "During Recession"
 
    else:
        return "After Recession"

unempl_recess['Indicator'] = unempl_recess['observation_date'].apply(lambda date : indicator(date))

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\1029144104.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
unempl_recess.tail()

,observation_date,Labor_Rate_Names,Labor_Rate_Values,State,DISCWORK,ALLMARGAT,CIVLF,EMPLOY,INVOLPTEMP,UNEMPLOY,UNEMP15,JOBLOS,unemployement_levels,Indicator
721,2014-01-01,All Employed Workers,92.984658,Florida,75700,154600,9483500,8818200,558900,665400,412400,376500,All Employed Workers,After Recession
1202,2014-01-01,Employed Involuntary Part-Time,6.716410,California,113100,312900,18629000,17031000,1251200,1598000,876600,853800,Unemployemnt Level 1,After Recession
1201,2014-01-01,All Employed Workers,91.421976,California,113100,312900,18629000,17031000,1251200,1598000,876600,853800,All Employed Workers,After Recession
1200,2014-01-01,All Marginally Attached Workers,1.679639,California,113100,312900,18629000,17031000,1251200,1598000,876600,853800,Unemployemnt Level 1,After Recession
1680,2014-01-01,All Marginally Attached Workers,1.161871,Texas,48800,149800,12893000,12123900,537000,769100,348800,348400,Unemployemnt Level 1,After Recession


In [12]:
unempl_recess_rate = unempl_recess[unempl_recess['Labor_Rate_Names'] != 'All Employed Workers']
empl_recess_rate = unempl_recess[unempl_recess['Labor_Rate_Names'] == 'All Employed Workers']
selectunemprate_unique = list(unempl_recess_rate['unemployement_levels'].unique())
selectstate_unique = list(unempl_recess_rate['State'].sort_values().unique())

Recession Line graph

In [13]:
select_state = dcc_dropdown(selectstate_unique,selectstate_unique[0],'state')
select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[1],'rate')
#creating graph components
unemp_rate = dcc.Graph(id = 'figure_1')

#callback setup

@app.callback(
        Output("figure_1", "figure"), 
        [Input("state", "value"),
        Input("rate", "value")]
)

def update_unemp_rate_graph(selected_state,selected_unemprate):
        
        unempl_recess_g =unempl_recess_rate[(unempl_recess['unemployement_levels'] == selected_unemprate) & (unempl_recess['State'] == selected_state) ] 
        unempl_recess_pre = unempl_recess_g[unempl_recess_g['Indicator'] == 'Before Recession']  
        unempl_recess_dur = unempl_recess_g[unempl_recess_g['Indicator'] == 'During Recession'] 
        unempl_recess_post = unempl_recess_g[unempl_recess_g['Indicator'] == 'After Recession']
        fig = make_subplots(rows=1, cols=3,shared_yaxes=True,subplot_titles=("Pre-Recession", "During-Recession", "Post-Recession"),x_title="Year",
                        y_title="Unemployment Rate")
        
        for type in unempl_recess_pre['Labor_Rate_Names'].unique():
                data_1 = unempl_recess_pre[unempl_recess_pre['Labor_Rate_Names'] == type]
                fig.add_trace(go.Scatter(x=data_1['observation_date'], y=data_1['Labor_Rate_Values'], mode='lines',name = type),
                                row=1, col=1)
        
        for type2 in unempl_recess_dur['Labor_Rate_Names'].unique():
                data_2 = unempl_recess_dur[unempl_recess_dur['Labor_Rate_Names'] == type2]
                fig.add_trace(go.Scatter(x=data_2['observation_date'], y=data_2['Labor_Rate_Values'], mode='lines',name = type2),
                                row=1, col=2)
        
        for type3 in unempl_recess_post['Labor_Rate_Names'].unique():
                data_3 = unempl_recess_post[unempl_recess_post['Labor_Rate_Names'] == type3]
                fig.add_trace(go.Scatter(x=data_3['observation_date'], y=data_3['Labor_Rate_Values'], mode='lines',name = type3),
                                row=1, col=3)
                
        fig.update_layout(title=selected_unemprate + ' in ' + selected_state, showlegend=True)
        return fig

Recession Bar Graph

In [14]:
select_state = dcc_dropdown(selectstate_unique,selectstate_unique[0],'state')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

unemp_bar = dcc.Graph(id = 'figure_2')

@app.callback(
        Output("figure_2", "figure"), 
        [Input("state", "value")]
)

def update_unemp_absolute_graph (selected_state):
        
        unempl_recess_g =empl_recess_rate[empl_recess_rate['State'] == selected_state] 
    
        unemp_absolute_graph_fig = px.bar(unempl_recess_g,
                  x = 'observation_date',
                  y = 'Labor_Rate_Values',
                  color = 'Indicator',
                  title=  'Employent Rate in ' + selected_state
                  )
        unemp_absolute_graph_fig.update_xaxes(title='Year')
        unemp_absolute_graph_fig.update_yaxes(title= 'Unemployment Rate')
        return unemp_absolute_graph_fig

Recession Tab

In [15]:
drop_down_3_tab = dbc.Card([
    card("Recession Analysis by state and Employment type"),
    dbc.Row([
           dbc.Col(dropdown("Select State",dcc_dropdown(selectstate_unique,selectstate_unique[0],'state')),width=5),
           dbc.Col(dropdown("Unemployment Levels",dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')),width=5)
                ]),
    dbc.Row([
           tab_graph("Unemployment Rate By State - Recession",unemp_rate),
           tab_graph("Employment Rate by State-Recession",unemp_bar)
    ])
])

DF cleaning for covid

In [16]:
unempl['observation_date'] = pd.to_datetime(unempl['observation_date'],format = '%d-%m-%Y')
unempl = unempl.sort_values('observation_date')
unempl_covid = unempl[(unempl['observation_date'] >= '2019') & (unempl['observation_date'] <= '2022')]
def indicator_covid(num):
    if num.year <= 2019:
        return "Before Covid"
 
    elif num.year >= 2019 and num.year < 2021:
        return "During Covid"
 
    else:
        return "After Covid"

unempl_covid['Indicator'] = unempl_covid['observation_date'].apply(lambda date : indicator_covid(date))
unempl_covid['Labor_Rate_Names'] = unempl_covid['Labor_Rate_Names'].replace(dic_names)
unempl_covid['unemployement_levels']=unempl_covid['Labor_Rate_Names'].apply(group)
unempl_covid.tail()

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\1723868552.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\1723868552.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\1723868552.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

,observation_date,Labor_Rate_Names,Labor_Rate_Values,State,DISCWORK,ALLMARGAT,CIVLF,EMPLOY,INVOLPTEMP,UNEMPLOY,UNEMP15,JOBLOS,unemployement_levels,Indicator
1873,2022-01-01,All Employed Workers,95.192087,Texas,34600,127400,14376300,13685100,419000,691200,294500,350200,All Employed Workers,After Covid
432,2022-01-01,All Marginally Attached Workers,1.517364,NewYork,48800,140300,9246300,8694500,309900,551800,322500,323700,Unemployemnt Level 1,After Covid
433,2022-01-01,All Employed Workers,94.032207,NewYork,48800,140300,9246300,8694500,309900,551800,322500,323700,All Employed Workers,After Covid
434,2022-01-01,Employed Involuntary Part-Time,3.351611,NewYork,48800,140300,9246300,8694500,309900,551800,322500,323700,Unemployemnt Level 1,After Covid
1872,2022-01-01,All Marginally Attached Workers,0.886181,Texas,34600,127400,14376300,13685100,419000,691200,294500,350200,Unemployemnt Level 1,After Covid


In [17]:
unempl_covid_rate = unempl_covid[unempl_covid['Labor_Rate_Names'] != 'All Employed Workers']
empl_covid_rate = unempl_covid[unempl_covid['Labor_Rate_Names'] == 'All Employed Workers']
selectunemprate_covid_unique = list(unempl_covid_rate['unemployement_levels'].unique())
selectstate_covid_unique = list(unempl_covid_rate['State'].sort_values().unique())

Line Graph for Covid

In [18]:
select_state_covid = dcc_dropdown(selectstate_covid_unique,selectstate_covid_unique[0],'state_covid')
select_unemprate_covid = dcc_dropdown(selectunemprate_covid_unique,selectunemprate_covid_unique[1],'rate_covid')
#creating graph components
unemp_rate_covid = dcc.Graph(id = 'figure_2_covidline')

#callback setup

@app.callback(
        Output("figure_2_covidline", "figure"), 
        [Input("state_covid", "value"),
        Input("rate_covid", "value")]
)

def update_unemp_rate_graph_covid(selected_state,selected_unemprate):
        
        unempl_covid_g =unempl_covid_rate[(unempl_covid['unemployement_levels'] == selected_unemprate) & (unempl_covid['State'] == selected_state) ] 
        unempl_covid_pre = unempl_covid_g[unempl_covid_g['Indicator'] == 'Before Covid']  
        unempl_covid_dur = unempl_covid_g[unempl_covid_g['Indicator'] == 'During Covid'] 
        unempl_covid_post = unempl_covid_g[unempl_covid_g['Indicator'] == 'After Covid']
        fig_line_covid= make_subplots(rows=1, cols=3,shared_yaxes=True,subplot_titles=("Pre-Covid", "During-Covid", "Post-Covid"),x_title="Year",
                        y_title="Unemployment Rate")
        
        for type in unempl_covid_pre['Labor_Rate_Names'].unique():
                data_1 = unempl_covid_pre[unempl_covid_pre['Labor_Rate_Names'] == type]
                fig_line_covid.add_trace(go.Scatter(x=data_1['observation_date'], y=data_1['Labor_Rate_Values'], mode='lines',name = type),
                                row=1, col=1)
        
        for type2 in unempl_covid_dur['Labor_Rate_Names'].unique():
                data_2 = unempl_covid_dur[unempl_covid_dur['Labor_Rate_Names'] == type2]
                fig_line_covid.add_trace(go.Scatter(x=data_2['observation_date'], y=data_2['Labor_Rate_Values'], mode='lines',name = type2),
                                row=1, col=2)
        
        for type3 in unempl_covid_post['Labor_Rate_Names'].unique():
                data_3 = unempl_covid_post[unempl_covid_post['Labor_Rate_Names'] == type3]
                fig_line_covid.add_trace(go.Scatter(x=data_3['observation_date'], y=data_3['Labor_Rate_Values'], mode='lines',name = type3),
                                row=1, col=3)
                
        fig_line_covid.update_layout(title=selected_unemprate + ' in ' + selected_state, showlegend=True)
        return fig_line_covid

Bar Graph for Covid

In [19]:
select_state_covid = dcc_dropdown(selectstate_covid_unique,selectstate_covid_unique[0],'state_covid')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

unemp_bar_covid = dcc.Graph(id = 'figure_2_covidbar')

@app.callback(
        Output("figure_2_covidbar", "figure"), 
        [Input("state_covid", "value")]
)

def update_unemp_absolute_graph_covid(selected_state):
        
        unempl_covid_g =empl_covid_rate[empl_covid_rate['State'] == selected_state] 
    
        employ_rate_covid = px.bar(unempl_covid_g,
                  x = 'observation_date',
                  y = 'Labor_Rate_Values',
                  color = 'Indicator',
                  title=  'Employent Rate in ' + selected_state + '(Covid)'
                  )
        employ_rate_covid.update_xaxes(title='Year')
        employ_rate_covid.update_yaxes(title='Employment Rate')
        return employ_rate_covid 

Covid tab

In [20]:
drop_down_4_tab = dbc.Card([
    card("Covid Analysis by state and Employment type"),
    dbc.Row([
           dbc.Col(dropdown("Select State",dcc_dropdown(selectstate_covid_unique,selectstate_covid_unique[0],'state_covid')),width=5),
           dbc.Col(dropdown("Unemployment Levels",dcc_dropdown(selectunemprate_covid_unique,selectunemprate_covid_unique[1],'rate_covid')),width=5)
                ]),
    dbc.Row([
           tab_graph("Unemployment Rate by State-Covid",unemp_rate_covid),
           tab_graph("Employment Rate by State-Covid",unemp_bar_covid)
    ])
    
])

In [21]:
unempl_covid.head()
unempl_recess.head()
emp_analysis = list(unempl_covid['Labor_Rate_Names'].unique())
emp_analysis
unempl_recess.head()

,observation_date,Labor_Rate_Names,Labor_Rate_Values,State,DISCWORK,ALLMARGAT,CIVLF,EMPLOY,INVOLPTEMP,UNEMPLOY,UNEMP15,JOBLOS,unemployement_levels,Indicator
0,2004-01-01,All Marginally Attached Workers,1.323962,NewYork,43000,124700,9418700,8809300,260600,609400,282300,345300,Unemployemnt Level 1,Before Recession
1443,2004-01-01,Civilians Unemployed for min 15 Weeks,2.289082,Texas,35300,106800,10978200,10252900,438400,725200,251300,368400,Unemployemnt Level 2,Before Recession
1442,2004-01-01,Employed Involuntary Part-Time,3.993369,Texas,35300,106800,10978200,10252900,438400,725200,251300,368400,Unemployemnt Level 1,Before Recession
1441,2004-01-01,All Employed Workers,93.393270,Texas,35300,106800,10978200,10252900,438400,725200,251300,368400,All Employed Workers,Before Recession
1440,2004-01-01,All Marginally Attached Workers,0.972837,Texas,35300,106800,10978200,10252900,438400,725200,251300,368400,Unemployemnt Level 1,Before Recession


Analysis Tab Recession Line Graph

In [22]:
emp_analysis_drop = dcc_dropdown(emp_analysis,emp_analysis[0],'emp_type_analysis')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

emp_analysis_graph = dcc.Graph(id = 'figure_1_analysis')

@app.callback(
        Output("figure_1_analysis", "figure"), 
        [Input("emp_type_analysis", "value")]
)

def update_emp_analysis_graph(selected_emp_type):
        
        unempl_recess_fig =unempl_recess[unempl_recess['Labor_Rate_Names'] == selected_emp_type] 
    
        employ_rate_recession_all = px.line(unempl_recess_fig,
                  x = 'observation_date',
                  y = 'Labor_Rate_Values',
                  color = 'State',
                  symbol = 'State',
                  title=  'State-wise ' + selected_emp_type + ' Comparision'
                  )
        employ_rate_recession_all.update_xaxes(title='Year')
        employ_rate_recession_all.update_yaxes(title= 'Unemployment Rate')
        return employ_rate_recession_all

Analysis Tab Covid Line Graph

In [23]:
emp_analysis_drop_covid = dcc_dropdown(emp_analysis,emp_analysis[0],'emp_type_analysis')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

emp_analysis_graph_covid = dcc.Graph(id = 'figure_2_analysis_covid')

@app.callback(
        Output("figure_2_analysis_covid", "figure"), 
        [Input("emp_type_analysis", "value")]
)

def update_emp_analysis_graph_covid(selected_emp_type):
        
        unempl_covid_fig =unempl_covid[unempl_covid['Labor_Rate_Names'] == selected_emp_type] 
    
        employ_rate_covid_all = px.line(unempl_covid_fig,
                  x = 'observation_date',
                  y = 'Labor_Rate_Values',
                  color = 'State',
                  symbol = 'State',
                  title=  'State-wise ' + selected_emp_type + ' Comparision'
                  )
        employ_rate_covid_all.update_xaxes(title='Year')
        employ_rate_covid_all.update_yaxes(title= 'Unemployment Rate')
        return employ_rate_covid_all

Analysis Recession Box Plot 

In [24]:
emp_analysis_drop = dcc_dropdown(emp_analysis,emp_analysis[0],'emp_type_analysis')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

emp_analysis_graph_box = dcc.Graph(id = 'figure_3_analysis_box')

@app.callback(
        Output("figure_3_analysis_box", "figure"), 
        [Input("emp_type_analysis", "value")]
)
def update_emp_analysis_graph_box(selected_emp_type):
        
        unempl_recess_fig_box =unempl_recess[unempl_recess['Labor_Rate_Names'] == selected_emp_type] 
        unempl_recess_fig_box = unempl_recess_fig_box[(unempl_recess_fig_box['observation_date'] >='2008') & (unempl_recess_fig_box['observation_date'] <='2010')]
        employ_rate_recession_box = px.box(unempl_recess_fig_box,
                  x = 'State',
                  y = 'Labor_Rate_Values',
                  color = 'State',
                  notched= True,
                  title=  'State-wise ' + selected_emp_type + ' (2008-2010)'
                  )
        employ_rate_recession_box.update_xaxes(title='Year')
        employ_rate_recession_box.update_yaxes(title= 'Unemployment Rate')
        return employ_rate_recession_box

In [25]:
test = unempl[(unempl['observation_date'] >='2020') & (unempl['observation_date'] <='2021')]
test.head()

,observation_date,Labor_Rate_Names,Labor_Rate_Values,State,DISCWORK,ALLMARGAT,CIVLF,EMPLOY,INVOLPTEMP,UNEMPLOY,UNEMP15,JOBLOS,unemployement_levels
865,2020-01-01,All Employed Workers,96.810373,Florida,31400,96700,10396200,10064600,311800,331600,127500,147100,All Employed Workers
864,2020-01-01,All Marginally Attached Workers,0.930148,Florida,31400,96700,10396200,10064600,311800,331600,127500,147100,Unemployemnt Level 1
867,2020-01-01,Civilians Unemployed for min 15 Weeks,1.226410,Florida,31400,96700,10396200,10064600,311800,331600,127500,147100,Unemployemnt Level 2
868,2020-01-01,Total Unemployment Workers,3.189627,Florida,31400,96700,10396200,10064600,311800,331600,127500,147100,Total Unemployment Workers
869,2020-01-01,Job losers,1.414940,Florida,31400,96700,10396200,10064600,311800,331600,127500,147100,Unemployemnt Level 2


Analysis Tab Covid box Plot

In [26]:
emp_analysis_drop = dcc_dropdown(emp_analysis,emp_analysis[0],'emp_type_analysis')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

emp_analysis_graph_box_covid = dcc.Graph(id = 'figure_4_analysis_box')

@app.callback(
        Output("figure_4_analysis_box", "figure"), 
        [Input("emp_type_analysis", "value")]
)
def update_emp_analysis_graph_box(selected_emp_type):
        
        unempl_covid_fig_box =unempl_covid[unempl_covid['Labor_Rate_Names'] == selected_emp_type] 
        unempl_covid_fig_box = unempl_covid_fig_box[(unempl_covid_fig_box['observation_date'] >='2020') & (unempl_covid_fig_box['observation_date'] <='2021')]
        employ_rate_covid_box = px.box(unempl_covid_fig_box,
                  x = 'State',
                  y = 'Labor_Rate_Values',
                  color = 'State',
                  notched= True,
                  title=  'State-wise ' + selected_emp_type + ' (2020-2021)'
                  )
        employ_rate_covid_box.update_xaxes(title='Year')
        employ_rate_covid_box.update_yaxes(title= 'Unemployment Rate')
        return employ_rate_covid_box

Dimension-1 Analysis Tab

In [27]:
drop_down_5_tab = dbc.Card([
    card("Unemployment Analysis by State (Recession Vs Covid)"),
    dbc.Row([
                dbc.Col(dropdown("Unemployment Levels",dcc_dropdown(emp_analysis,emp_analysis[0],'emp_type_analysis')),width=5)
                ]),
    dbc.Row([
                dbc.Col(html.Div(tab_graph("Unemployment Type Comparision(Recession)",emp_analysis_graph)),width=6),
                dbc.Col(html.Div(tab_graph("Unemployment Type Comparision(Covid)",emp_analysis_graph_covid)),width=6)
    ]),
    dbc.Row([
                dbc.Col(html.Div(tab_graph("Unemployment Median Comparision(Recession)",emp_analysis_graph_box)),width=6),
                dbc.Col(html.Div(tab_graph("Unemployment Median Comparision(Covid)",emp_analysis_graph_box_covid)),width=6)
    ])
])

## Housing Dimention

In [28]:
housing_hpi = pd.read_csv('Housing_Output_Hpi.csv')
housing_tier = pd.read_csv('Housing_Output 2_top_bottom.csv')
housing_hpi['observation_date'] = pd.to_datetime(housing_hpi['observation_date'],format = '%d-%m-%Y')
housing_hpi = housing_hpi.sort_values('observation_date')
housing_tier['observation_date'] = pd.to_datetime(housing_tier['observation_date'],format = '%d-%m-%Y')
housing_tier = housing_tier.sort_values('observation_date')
#housing_hpi.tail(10)
#housing_tier.head()
housing_tier.rename(columns = {'RegionName':'State'},inplace=True)
housing_tier['Housing Types'] = housing_tier['Housing Types'].replace({'Top Quantile House Vales':'Top Quantile House Price',
                                                                       'Bottom Quantile House Values':'Bottom Quantile House Price'})
housing_hpi['State'] = housing_hpi['State'].replace({'NewYork':'New York'})
housing_tier.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2312 entries, 1397 to 2023
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   observation_date  2312 non-null   datetime64[ns]
 1   State             2312 non-null   object        
 2   Housing Types     2312 non-null   object        
 3   House Values      2312 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 90.3+ KB


In [29]:
housing_hpi.head()
#housing_tier.head()

,observation_date,State,Housing,Housing Values
0,2000-01-01,California,HPI,260.97000
11,2000-01-01,Texas,Total Personal income,575999.00000
10,2000-01-01,Texas,Housing Permits,10623.76652
9,2000-01-01,Texas,HPI,158.95000
7,2000-01-01,New York,Housing Permits,2246.00000


In [30]:
housing_hpi_recess = housing_hpi[(housing_hpi['observation_date'] >= '2004') & (housing_hpi['observation_date'] <= '2014')]
housing_tier_recess = housing_tier[(housing_tier['observation_date'] >= '2004') & (housing_tier['observation_date'] <= '2014')]
def indicator(num):
        if num.year <= 2007:
            return "Pre Recession"
 
        elif num.year >= 2008 and num.year < 2012:
            return "During Recession"
 
        else:
            return "Post Recession"

housing_hpi_recess['Indicator'] = housing_hpi_recess['observation_date'].apply(lambda date : indicator(date))
housing_tier_recess['Indicator'] = housing_tier_recess['observation_date'].apply(lambda date : indicator(date))

housing_hpi_recess.tail()
housing_tier_recess.head()

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\4209488860.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\4209488860.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,observation_date,State,Housing Types,House Values,Indicator
1405,2004-01-31,Florida,Top Quantile House Price,307123.00000,Pre Recession
1404,2004-01-31,Florida,Bottom Quantile House Price,77814.93308,Pre Recession
1983,2004-01-31,New York,Top Quantile House Price,505443.66320,Pre Recession
1982,2004-01-31,New York,Bottom Quantile House Price,92969.27195,Pre Recession
826,2004-01-31,Texas,Bottom Quantile House Price,62464.88463,Pre Recession


In [31]:
#selectunemprate_unique = list(unempl_recess_rate['unemployement_levels'].unique())
housing_state = list(housing_hpi_recess['State'].sort_values().unique())
#housing_tier_recess['State'].sort_values().unique() #== housing_hpi_recess['State'].sort_values().unique()
list(housing_hpi_recess['Indicator'].sort_values().unique())[1]

'Post Recession'

Function for Creating Line graph

In [32]:
def line_graph_house(df,selected_state,cat_column,x,y):    
    df =df[(df['State'] == selected_state)] 
    df_pre = df[df['Indicator'] == list(df['Indicator'].sort_values().unique())[2]]  
    df_dur = df[df['Indicator'] == list(df['Indicator'].sort_values().unique())[0]] 
    df_post = df[df['Indicator'] == list(df['Indicator'].sort_values().unique())[1]]
    fig = make_subplots(rows=1, cols=3,shared_yaxes=True,subplot_titles=(list(df['Indicator'].sort_values().unique())[2],list(df['Indicator'].sort_values().unique())[0],list(df['Indicator'].sort_values().unique())[1]),x_title="Year",
                        y_title="House Price")
        
    for type in df_pre[cat_column].unique():
        data_1 = df_pre[df_pre[cat_column] == type]
        fig.add_trace(go.Scatter(x=data_1[x], y=data_1[y], mode='lines',name = type),
                                row=1, col=1)
        
    for type2 in df_dur[cat_column].unique():
        data_2 = df_dur[df_dur[cat_column] == type2]
        fig.add_trace(go.Scatter(x=data_2[x], y=data_2[y], mode='lines',name = type2),
                                row=1, col=2)
        
    for type3 in df_post[cat_column].unique():
        data_3 = df_post[df_post[cat_column] == type3]
        fig.add_trace(go.Scatter(x=data_3[x], y=data_3[y], mode='lines',name = type3),
                                row=1, col=3)
    fig.update_layout(title='House Price values' + ' in ' + selected_state, showlegend=True)            
    
    return fig

In [33]:
line_graph_house(housing_tier_recess,
                 'Texas',
                 'Housing Types',
                 'observation_date',
                 'House Values')

Function for creating Bar graph Home dimention 

In [34]:
def bar_graph_house(df,selected_state,cat,x,y):
      df =df[(df['State'] == selected_state) & (df['Housing'] == cat)] 
      df_fig = px.bar(df,
                  x = x,
                  y = y,
                  color = 'Indicator',
                  title= 'Housing Permits' +'in ' + selected_state
                  )
      df_fig.update_layout(   xaxis=dict(title='Year' ),
                              yaxis=dict(title='Number of Housing Permits')
      )
      return df_fig

Function for creating Line graph_hpi Home dimention 

In [35]:
def hpiline_graph_house(df,selected_state,cat,x,y):
      df =df[(df['State'] == selected_state) & (df['Housing'] == cat)] 
      df_fig = px.line(df,
                  x = x,
                  y = y,
                  color = 'Indicator',
                  symbol= 'Indicator',
                  title= 'House Price Index ' +'in ' + selected_state
                  )
      df_fig.update_layout(   xaxis=dict(title='Year' ),
                              yaxis=dict(title='House Price Index')
      )
      return df_fig

In [36]:
hpiline_graph_house(housing_hpi_recess,
                'Texas',
                'HPI',
                'observation_date',
                'Housing Values')

In [37]:
bar_graph_house(housing_hpi_recess,
                'Texas',
                'Housing Permits',
                'observation_date',
                'Housing Values')

Recessoin Line Graph

In [38]:
select_housing_state = dcc_dropdown(housing_state,housing_state[0],'Home_Recession')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_home_recess = dcc.Graph(id = 'figure_line_recess')

@app.callback(
        Output("figure_line_recess", "figure"), 
        [Input("Home_Recession", "value")]
)
def update_line_home_recess(selected_state):
    return line_graph_house(housing_tier_recess,
                 selected_state,
                 'Housing Types',
                 'observation_date',
                 'House Values')

Recession Bar Graph

In [39]:
select_housing_state = dcc_dropdown(housing_state,housing_state[0],'Home_Recession')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

bar_home_recess = dcc.Graph(id = 'figure_bar_recess')

@app.callback(
        Output("figure_bar_recess", "figure"), 
        [Input("Home_Recession", "value")]
)
def update_bar_home_recess(selected_state):
    return bar_graph_house(housing_hpi_recess,
                selected_state,
                'Housing Permits',
                'observation_date',
                'Housing Values')

Recession HPI Line graph

In [40]:
select_housing_state = dcc_dropdown(housing_state,housing_state[0],'Home_Recession')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_hpi_home_recess = dcc.Graph(id = 'figure_Line_hpi_recess')

@app.callback(
        Output("figure_Line_hpi_recess", "figure"), 
        [Input("Home_Recession", "value")]
)
def update_line_hpi_home_recess(selected_state):
    return hpiline_graph_house(housing_hpi_recess,
                selected_state,
                'HPI',
                'observation_date',
                'Housing Values')

Home recession Tab

In [41]:
drop_down_6_tab = dbc.Card([
    card("Home Dimention Analysis(recession)"),
    dbc.Row([
           dbc.Col(dropdown("Select State",dcc_dropdown(housing_state,housing_state[0],'Home_Recession')),width=5)
                ]),
    dbc.Row([
           tab_graph("Home Top/Bottom Tire Prices by State",line_home_recess),
    ]),
    dbc.Row([
                dbc.Col(html.Div(tab_graph("House Price State (Recession)",line_hpi_home_recess)),width=6),
                dbc.Col(html.Div(tab_graph("Hosuing Permits by State (Recession)",bar_home_recess)),width=6)
    ])
])

In [42]:
housing_hpi_covid = housing_hpi[(housing_hpi['observation_date'] >= '2019') & (housing_hpi['observation_date'] <= '2022')]
housing_tier_covid = housing_tier[(housing_tier['observation_date'] >= '2019') & (housing_tier['observation_date'] <= '2022')]
def indicator(num):
        if num.year <= 2019:
            return "Pre Covid"
 
        elif num.year >= 2019 and num.year < 2021:
            return "During Covid"
 
        else:
            return "Post Covid"

housing_hpi_covid['Indicator'] = housing_hpi_covid['observation_date'].apply(lambda date : indicator(date))
housing_tier_covid['Indicator'] = housing_tier_covid['observation_date'].apply(lambda date : indicator(date))

housing_hpi_covid.tail()
housing_tier_covid.head()

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\1430797336.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\1430797336.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,observation_date,State,Housing Types,House Values,Indicator
1434,2019-01-31,Florida,Bottom Quantile House Price,1.402707e+05,Pre Covid
1435,2019-01-31,Florida,Top Quantile House Price,4.090990e+05,Pre Covid
2012,2019-01-31,New York,Bottom Quantile House Price,1.384782e+05,Pre Covid
279,2019-01-31,California,Top Quantile House Price,1.003007e+06,Pre Covid
278,2019-01-31,California,Bottom Quantile House Price,3.115810e+05,Pre Covid


Covid Line Graph

In [43]:
select_housing_state = dcc_dropdown(housing_state,housing_state[0],'Home_covid')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_home_covid = dcc.Graph(id = 'figure_line_covid')

@app.callback(
        Output("figure_line_covid", "figure"), 
        [Input("Home_covid", "value")]
)
def update_line_home_covid(selected_state):
    return line_graph_house(housing_tier_covid,
                 selected_state,
                 'Housing Types',
                 'observation_date',
                 'House Values')

Covid bar graph

In [44]:
select_housing_state = dcc_dropdown(housing_state,housing_state[0],'Home_covid')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

bar_home_covid = dcc.Graph(id = 'figure_bar_covid')

@app.callback(
        Output("figure_bar_covid", "figure"), 
        [Input("Home_covid", "value")]
)
def update_bar_home_covid(selected_state):
    return bar_graph_house(housing_hpi_covid,
                selected_state,
                'Housing Permits',
                'observation_date',
                'Housing Values')

Covid HPI line Graph

In [45]:
select_housing_state = dcc_dropdown(housing_state,housing_state[0],'Home_covid')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_hpi_home_covid = dcc.Graph(id = 'figure_Line_hpi_covid')

@app.callback(
        Output("figure_Line_hpi_covid", "figure"), 
        [Input("Home_covid", "value")]
)
def update_line_hpi_home_covid(selected_state):
    return hpiline_graph_house(housing_hpi_covid,
                selected_state,
                'HPI',
                'observation_date',
                'Housing Values')

Home Covid tab

In [46]:
drop_down_7_tab = dbc.Card([
    card("Home Dimention Analysis(Covid)"),
    dbc.Row([
           dbc.Col(dropdown("Select State",dcc_dropdown(housing_state,housing_state[0],'Home_covid')),width=5)
                ]),
    dbc.Row([
           tab_graph("Home Top/Bottom Tire Prices by State",line_home_covid),
    ]),
    dbc.Row([
                dbc.Col(html.Div(tab_graph("House Price State (Covid)",line_hpi_home_covid)),width=6),
                dbc.Col(html.Div(tab_graph("Hosuing Permits by State (Covid)",bar_home_covid)),width=6)
    ])
])

Tab Home Analysis

In [47]:
def home_analy_line(df,select_home_type,cat,x,y):
    df =df[df[cat] == select_home_type] 
    
    anay_all = px.line(df,
                  x = x,
                  y = y,
                  color = 'State',
                  symbol = 'State',
                  title=  'State-wise ' + select_home_type + ' Comparision'
                  )
    return anay_all

In [48]:
home_analy_line(housing_tier_recess,'Top Quantile House Price','Housing Types','observation_date','House Values')
#housing_tier_recess['Housing Types'].unique()

In [49]:
def home_analy_box(df,year1,year2,select_cat):
     df = df[(df['observation_date'] >= year1) & (df['observation_date'] <= year2)]
     df = df[df['Housing']== select_cat]
     analy_box = px.box(df,
                  x = 'State',
                  y = 'Housing Values',
                  color = 'State',
                  notched= True,
                  title=  'State-wise ' + select_cat + '('+ year1 +'-'+ year2+')'
                  )
     return analy_box

In [50]:
home_analy_box(housing_hpi,'2020','2021','Housing Permits')

Recess Analysis Line graph

In [51]:
housing_type = list(housing_tier_recess['Housing Types'].unique())

select_housing_Type = dcc_dropdown(housing_type,housing_type[0],'Home_type')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_home_analy = dcc.Graph(id = 'figure_Line_analy')

@app.callback(
        Output('figure_Line_analy', "figure"), 
        [Input("Home_type", "value")]
)
def update_line_home_analy(selected_type):
    return home_analy_line(housing_tier_recess,
                           selected_type,
                           'Housing Types',
                           'observation_date',
                           'House Values')

Covid Analysis Line graph

In [52]:
housing_type = list(housing_tier_recess['Housing Types'].unique())

select_housing_Type = dcc_dropdown(housing_type,housing_type[0],'Home_type')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_home_analy_covid = dcc.Graph(id = 'figure_Line_analy_covid')

@app.callback(
        Output('figure_Line_analy_covid', "figure"), 
        [Input("Home_type", "value")]
)
def update_home_analy_covid(selected_type):
    return home_analy_line(housing_tier_covid,
                           selected_type,
                           'Housing Types',
                           'observation_date',
                           'House Values')

In [53]:
box_home_analy = dcc.Graph(id = 'figure_box_analy')
housing_type = list(housing_tier_recess['Housing Types'].unique())

select_housing_Type = dcc_dropdown(housing_type,housing_type[0],'Home_type')
@app.callback(
        Output('figure_box_analy', "figure"), 
        [Input("Home_type", "value")]
)
def box(x):
    return home_analy_box(housing_hpi,
                          '2008',
                          '2010',
                          'Housing Permits')

In [54]:
box_home_analy_covid = dcc.Graph(id = 'figure_box_analy_covid')
housing_type = list(housing_tier_recess['Housing Types'].unique())

select_housing_Type = dcc_dropdown(housing_type,housing_type[0],'Home_type')
@app.callback(
        Output('figure_box_analy_covid', "figure"), 
        [Input("Home_type", "value")]
)
def box(x):
    return home_analy_box(housing_hpi,
                          '2020',
                          '2021',
                          'Housing Permits')

Analysis Tab Housing

In [55]:
drop_down_8_tab = dbc.Card([
    card("Housing Analysis by State (Recession Vs Covid)"),
    dbc.Row([
                dbc.Col(dropdown("Housing Type",dcc_dropdown(housing_type,housing_type[0],'Home_type')),width=5)
                ]),
    dbc.Row([
                dbc.Col(html.Div(tab_graph("Housing Type Comparision(Recession)",line_home_analy)),width=6),
                dbc.Col(html.Div(tab_graph("Housing Type Comparision(Covid)",line_home_analy_covid)),width=6)
    ]),
    dbc.Row([
                dbc.Col(html.Div(tab_graph("Housing Permits Median Comparision(Recession)",box_home_analy)),width=6),
                dbc.Col(html.Div(tab_graph("Housing Permits Median Comparision(Covid)",box_home_analy_covid)),width=6)
    ])
])

## Economic Dimention

In [56]:
#selectunemprate_unique = list(unempl_recess_rate['unemployement_levels'].unique())
economic_dim = pd.read_csv('Output_Economic_well_being.csv')
economic_dim['observation_date'] = pd.to_datetime(economic_dim['observation_date'],format = '%d-%m-%Y')
economic_dim = economic_dim.sort_values('observation_date')
economic_dim.info()

economic_dim_recess = economic_dim[(economic_dim['observation_date'] >= '2005') & (economic_dim['observation_date'] <= '2014')]
def indicator(num):
        if num.year <= 2006:
            return "Pre Recession"
 
        elif num.year >= 2007 and num.year < 2010:
            return "During Recession"
 
        else:
            return "Post Recession"
        
economic_dim_recess['Indicator'] = economic_dim_recess['observation_date'].apply(lambda date : indicator(date))

def indicator_1(num):
        if num.year <= 2017:
            return "Pre Covid"
 
        elif num.year >= 2018 and num.year < 2021:
            return "During Covid"
 
        else:
            return "Post Covid"
economic_dim_covid = economic_dim[(economic_dim['observation_date'] >= '2016') & (economic_dim['observation_date'] <= '2022')]
economic_dim_covid['Indicator'] = economic_dim_covid['observation_date'].apply(lambda date : indicator_1(date))


<class 'pandas.core.frame.DataFrame'>
Index: 368 entries, 0 to 367
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   observation_date    368 non-null    datetime64[ns]
 1   State               368 non-null    object        
 2   Economic Indicator  368 non-null    object        
 3   Value               368 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 14.4+ KB


C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\273446102.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\273446102.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
economic_dim_recess.tail()
#economic_dim_covid.head()

,observation_date,State,Economic Indicator,Value,Indicator
227,2014-01-01,California,Total Personal Income,1902645.1,Post Recession
226,2014-01-01,California,Per Capita Per Person,50619.0,Post Recession
225,2014-01-01,California,Minimum Wage,9.0,Post Recession
224,2014-01-01,California,Civil Labor Force,18569356.0,Post Recession
231,2014-01-01,Florida,Total Personal Income,822886.5,Post Recession


Function for all state line plot for Economy dimention

In [58]:
def line_graph_econ_1(df,selected_ecom_type,cat_column,x,y):    
    df =df[(df['Economic Indicator'] == selected_ecom_type)] 
    df_pre = df[df['Indicator'] == list(df['Indicator'].sort_values().unique())[2]]  
    df_dur = df[df['Indicator'] == list(df['Indicator'].sort_values().unique())[0]] 
    df_post = df[df['Indicator'] == list(df['Indicator'].sort_values().unique())[1]]
    fig = make_subplots(rows=1, cols=3,shared_yaxes=True,subplot_titles=(list(df['Indicator'].sort_values().unique())[2],list(df['Indicator'].sort_values().unique())[0],list(df['Indicator'].sort_values().unique())[1]),x_title="Year",
                        y_title= selected_ecom_type)
    
    colors = {'New York': 'blue', 
               'Texas': 'red', 
               'California': 'green',
              'Florida': 'orange'}
        
    for type in df_pre[cat_column].unique():
        data_1 = df_pre[df_pre[cat_column] == type]
        fig.add_trace(go.Scatter(x=data_1[x], y=data_1[y], mode='lines',name = type,line=dict(color=colors[type]),showlegend=False),
                      row=1, col=1)
        
    for type2 in df_dur[cat_column].unique():
        data_2 = df_dur[df_dur[cat_column] == type2]
        fig.add_trace(go.Scatter(x=data_2[x], y=data_2[y], mode='lines',name = type2,line=dict(color=colors[type2]),showlegend=False),
                                row=1, col=2)
        
    for type3 in df_post[cat_column].unique():
        data_3 = df_post[df_post[cat_column] == type3]
        fig.add_trace(go.Scatter(x=data_3[x], y=data_3[y], mode='lines',name = type3,line=dict(color=colors[type3]),showlegend=False),
                                row=1, col=3)
    
    for state, color in colors.items():
        fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=state, line=dict(color=color)), row=1, col=1)
    
    # Add legend for the first subplot
    fig.update_layout(legend=dict(x=10, y=1, orientation="v"))
    
    fig.update_layout(title=' State-Wise ' + selected_ecom_type, showlegend=True)            
    
    return fig

In [59]:
economic_dim_recess

,observation_date,State,Economic Indicator,Value,Indicator
89,2005-01-01,New York,Minimum Wage,6.00,Pre Recession
95,2005-01-01,Texas,Total Personal Income,732005.30,Pre Recession
94,2005-01-01,Texas,Per Capita Per Person,33136.00,Pre Recession
93,2005-01-01,Texas,Minimum Wage,5.15,Pre Recession
92,2005-01-01,Texas,Civil Labor Force,11104720.00,Pre Recession
...,...,...,...,...,...
227,2014-01-01,California,Total Personal Income,1902645.10,Post Recession
226,2014-01-01,California,Per Capita Per Person,50619.00,Post Recession
225,2014-01-01,California,Minimum Wage,9.00,Post Recession
224,2014-01-01,California,Civil Labor Force,18569356.00,Post Recession


function for bat plot economy dimention

In [60]:
def bar_graph_econ(df,selected_state,cat,x,y):
      df =df[(df['State'] == selected_state) & (df['Economic Indicator'] == cat)] 
      df_fig = px.bar(df,
                  x = x,
                  y = y,
                  color = 'Indicator',
                  title= 'Minimum Wage ' +'in ' + selected_state
                  )
      df_fig.update_layout(   xaxis=dict(title='Year' ),
                              yaxis=dict(title='Minimum Wage')
      )
      return df_fig

In [61]:
bar_graph_econ(economic_dim_recess,
               'Florida',
               'Minimum Wage',
               'observation_date',
               'Value')

Line grpah for total personal income

In [62]:
def line_graph_econ(df,selected_state,cat,x,y):
      df =df[(df['State'] == selected_state) & (df['Economic Indicator'] == cat)] 
      df_fig = px.line(df,
                  x = x,
                  y = y,
                  color = 'Indicator',
                  symbol= 'Indicator',
                  title= cat +' in ' + selected_state
                  )
      df_fig.update_layout(   xaxis=dict(title='Year' ),
                              yaxis=dict(title=cat)
      )
      return df_fig

In [63]:
line_graph_econ(economic_dim_recess,
                'Texas',
                'Total Personal Income',
               'observation_date',
               'Value')

In [64]:
def econ_analy_box(df,selected_state,cat):
     df = df[(df['State'] == selected_state) & (df['Economic Indicator'] == cat)]
     analy_box = px.box(df,
                  x = 'Indicator',
                  y = 'Value',
                  color = 'Indicator',
                  notched= True,
                  title=  'Civil Labor Force ' +'in ' + selected_state
                  )
     return analy_box

In [65]:
econ_analy_box(economic_dim_recess,
               'New York',
               'Civil Labor Force')

Econimic Recession Analysis Graph Line

In [66]:
economic_indi_drop = list(economic_dim_recess['Economic Indicator'].sort_values().unique())

select_econ_Type_recess = dcc_dropdown(economic_indi_drop,economic_indi_drop[0],'econ_type')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_all_econ_recess = dcc.Graph(id = 'figure_Line_econ_recess')

@app.callback(
        Output('figure_Line_econ_recess', "figure"), 
        [Input("econ_type", "value")]
)
def update_line_all_econ_recess(selected_type):
    return line_graph_econ_1(economic_dim_recess,
                selected_type,
                'State',
                'observation_date',
                'Value')

In [67]:
economic_indi_drop

['Civil Labor Force',
 'Minimum Wage',
 'Per Capita Per Person',
 'Total Personal Income']

Total Personal Income line Graph

In [68]:
def line_graph_econ(df,selected_state,cat,x,y):
      df =df[(df['State'] == selected_state) & (df['Economic Indicator'] == cat)] 
      df_fig = px.line(df,
                  x = x,
                  y = y,
                  color = 'Indicator',
                  symbol= 'Indicator',
                  title= cat +' in ' + selected_state
                  )
      df_fig.update_layout(   xaxis=dict(title='Year' ),
                              yaxis=dict(title=cat)
      )
      return df_fig

In [69]:
economic_state_drop = list(economic_dim_recess['State'].sort_values().unique())

select_econ_state_recess = dcc_dropdown(economic_state_drop,economic_state_drop[0],'econ_state')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_state_econ_recess = dcc.Graph(id = 'figure_Line_econ_recess_state')

@app.callback(
        Output('figure_Line_econ_recess_state', "figure"), 
        [Input("econ_state", "value")]
)
def update_line_state_econ_recess(selected_type):
    return line_graph_econ(economic_dim_recess,
                selected_type,
                'Total Personal Income',
               'observation_date',
               'Value')

bar chart for economic recession MInimum wage

In [70]:
economic_state_drop = list(economic_dim_recess['State'].sort_values().unique())

select_econ_state_recess = dcc_dropdown(economic_state_drop,economic_state_drop[0],'econ_state')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

bar_state_econ_recess = dcc.Graph(id = 'figure_bar_econ_recess_state')

@app.callback(
        Output('figure_bar_econ_recess_state', "figure"), 
        [Input("econ_state", "value")]
)
def update_bar_state_econ_recess(selected_type):
    return bar_graph_econ(economic_dim_recess,
               selected_type,
               'Minimum Wage',
               'observation_date',
               'Value')

Box plot for economic labor force

In [71]:
economic_state_drop = list(economic_dim_recess['State'].sort_values().unique())

select_econ_state_recess = dcc_dropdown(economic_state_drop,economic_state_drop[0],'econ_state')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

box_state_econ_recess = dcc.Graph(id = 'figure_box_econ_recess_state')

@app.callback(
        Output('figure_box_econ_recess_state', "figure"), 
        [Input("econ_state", "value")]
)
def update_bar_state_econ_recess(selected_type):
    return econ_analy_box(economic_dim_recess,
               selected_type,
               'Civil Labor Force')

In [72]:
economic_state_drop = list(economic_dim_recess['State'].sort_values().unique())

select_econ_state_recess = dcc_dropdown(economic_state_drop,economic_state_drop[0],'econ_state')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_extra_state_econ_recess = dcc.Graph(id = 'figure_Line_extra_econ_recess_state')

@app.callback(
        Output('figure_Line_extra_econ_recess_state', "figure"), 
        [Input("econ_state", "value")]
)
def update_line_extra_state_econ_recess(selected_type):
    return line_graph_econ(economic_dim_recess,
                selected_type,
                'Civil Labor Force',
               'observation_date',
               'Value')

Economy Recession tab

In [73]:
drop_down_9_tab = dbc.Card([
    card("Economic Well-Being Recession Analysis"),
    dbc.Row([
                dbc.Col(dropdown("Economic Indicator",dcc_dropdown(economic_indi_drop,economic_indi_drop[0],'econ_type')),width=5)
                ]),
    dbc.Row([
                dbc.Col(tab_graph("Economic Well Being Comparision by State",line_all_econ_recess ))
    ]),
    dbc.Row([
                dbc.Col(dropdown("Select a State",dcc_dropdown(economic_state_drop,economic_state_drop[0],'econ_state')),width=5)
                ]),
    dbc.Row([
                dbc.Col(html.Div(tab_graph("Total Personal Income Comparision (Recession)",line_state_econ_recess)),width=6),
                dbc.Col(html.Div(tab_graph("Minimum Wage Comparision(Recession)",bar_state_econ_recess)),width=6),
                dbc.Col(html.Div(tab_graph("Civil Labor Force Median Comparision(Recession)",box_state_econ_recess)),width=6),
                dbc.Col(html.Div(tab_graph("Civil Labor Force Comparision(Recession)",line_extra_state_econ_recess)),width=6)
    ])
])

In [74]:
economic_dim_covid

,observation_date,State,Economic Indicator,Value,Indicator
271,2016-01-01,Texas,Total Personal Income,1262423.30,Pre Covid
270,2016-01-01,Texas,Per Capita Per Person,45862.00,Pre Covid
269,2016-01-01,Texas,Minimum Wage,7.25,Pre Covid
268,2016-01-01,Texas,Civil Labor Force,13224910.00,Pre Covid
267,2016-01-01,New York,Total Personal Income,1165397.60,Pre Covid
...,...,...,...,...,...
353,2022-01-01,California,Minimum Wage,14.00,Post Covid
352,2022-01-01,California,Civil Labor Force,19200001.00,Post Covid
366,2022-01-01,Texas,Per Capita Per Person,62586.00,Post Covid
358,2022-01-01,Florida,Per Capita Per Person,64806.00,Post Covid


Line graph covid - Economic

In [75]:
economic_indi_drop_covid = list(economic_dim_covid['Economic Indicator'].sort_values().unique())

select_econ_Type_covid = dcc_dropdown(economic_indi_drop_covid,economic_indi_drop_covid[0],'econ_type_covid')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_all_econ_covid = dcc.Graph(id = 'figure_Line_econ_covid')

@app.callback(
        Output('figure_Line_econ_covid', "figure"), 
        [Input("econ_type_covid", "value")]
)
def update_line_all_econ_covid(selected_type):
    return line_graph_econ_1(economic_dim_covid,
                selected_type,
                'State',
                'observation_date',
                'Value')

Covid line graph Total personal Income

In [76]:
economic_state_drop_covid = list(economic_dim_covid['State'].sort_values().unique())

select_econ_state_covid = dcc_dropdown(economic_state_drop_covid,economic_state_drop_covid[0],'econ_state_covid')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_state_econ_covid = dcc.Graph(id = 'figure_Line_econ_covid_state')

@app.callback(
        Output('figure_Line_econ_covid_state', "figure"), 
        [Input("econ_state_covid", "value")]
)
def update_line_state_econ_covid(selected_type):
    return line_graph_econ(economic_dim_covid,
                selected_type,
                'Total Personal Income',
               'observation_date',
               'Value')

Covid bar plot

In [77]:
economic_state_drop_covid = list(economic_dim_covid['State'].sort_values().unique())

select_econ_state_covid = dcc_dropdown(economic_state_drop_covid,economic_state_drop_covid[0],'econ_state_covid')
#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

bar_state_econ_covid = dcc.Graph(id = 'figure_bar_econ_covid_state')

@app.callback(
        Output('figure_bar_econ_covid_state', "figure"), 
        [Input("econ_state_covid", "value")]
)
def update_bar_state_econ_covid(selected_type):
    return bar_graph_econ(economic_dim_covid,
               selected_type,
               'Minimum Wage',
               'observation_date',
               'Value')

covid box plot

In [78]:
economic_state_drop_covid = list(economic_dim_covid['State'].sort_values().unique())

select_econ_state_covid = dcc_dropdown(economic_state_drop_covid,economic_state_drop_covid[0],'econ_state_covid')

#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

box_state_econ_covid = dcc.Graph(id = 'figure_box_econ_covid_state')

@app.callback(
        Output('figure_box_econ_covid_state', "figure"), 
        [Input("econ_state_covid", "value")]
)
def update_bar_state_econ_covid(selected_type):
    return econ_analy_box(economic_dim_covid,
               selected_type,
               'Civil Labor Force')

Line Graph Extra Covid

In [79]:
economic_state_drop_covid = list(economic_dim_covid['State'].sort_values().unique())

select_econ_state_covid = dcc_dropdown(economic_state_drop_covid,economic_state_drop_covid[0],'econ_state_covid')

#select_unemprate = dcc_dropdown(selectunemprate_unique,selectunemprate_unique[0],'rate')

line_extra_state_econ_covid = dcc.Graph(id = 'figure_Line_extra_econ_covid_state')

@app.callback(
        Output('figure_Line_extra_econ_covid_state', "figure"), 
        [Input("econ_state_covid", "value")]
)
def update_line_extra_state_econ_covid(selected_type):
    return line_graph_econ(economic_dim_covid,
                selected_type,
                'Civil Labor Force',
               'observation_date',
               'Value')

Economy_Covid Tab

In [80]:
drop_down_10_tab = dbc.Card([
    card("Economic Well-Being Covid Analysis"),
    dbc.Row([
                dbc.Col(dropdown("Economic Indicator",dcc_dropdown(economic_indi_drop_covid,economic_indi_drop_covid[0],'econ_type_covid')
),width=5)
                ]),
    dbc.Row([
                dbc.Col(tab_graph("Economic Well Being Comparision by State",line_all_econ_covid ))
    ]),
    dbc.Row([
                dbc.Col(dropdown("Select a State",dcc_dropdown(economic_state_drop_covid,economic_state_drop_covid[0],'econ_state_covid')
),width=5)
                ]),
    dbc.Row([
                dbc.Col(html.Div(tab_graph("Total Personal Income Comparision (Covid)",line_state_econ_covid)),width=6),
                dbc.Col(html.Div(tab_graph("Minimum Wage Comparision(Covid)",bar_state_econ_covid)),width=6),
                dbc.Col(html.Div(tab_graph("Civil Labor Force Median Comparision(Covid)",box_state_econ_covid)),width=6),
                dbc.Col(html.Div(tab_graph("Civil Labor Force Comparision(Covid)",line_extra_state_econ_covid)),width=6)
    ])
])

Introduction : Unemployement

In [81]:
worker_groups = [
    {
        'name':'Introduction',
        'description':'Impacts of Great Recession (Dec 2007 to Jun 2009) and the COVID-19 pandemic in 2020 in Aspect of unemployment in California, Florida, New York, and Texas.This Analysis provide insigts of three categories of unemployment Level in these states.'
    },
    {
        "name": "Unemployment Level 1",
        "description": "This group comprises workers classified as 'All Marginally Attached Workers' and 'Employed Involuntary Part-Time.' These individuals are often on the fringe of the labor force, facing challenges such as underemployment or involuntary part-time work. Despite being employed, they may not have secure or full-time positions, contributing to the complexities of measuring unemployment rates accurately."
    },
    {
        "name": "Unemployment Level 2",
        "description": "The second level of unemployment encompasses workers identified as 'Civilians Unemployed for min 15 Weeks' and 'Job losers.' These individuals have experienced significant disruptions in their employment, including extended periods of unemployment or recent job losses. They represent a segment of the workforce grappling with job market transitions, seeking new opportunities, or facing economic hardships due to layoffs or restructuring."
    },
    {
        "name": "Other Workers",
        "description": "Apart from the categorized groups, the dataset may include various other labor rate names representing different segments of the workforce. These could include 'Total Unemployment Workers,' 'All Employed Workers,' and other specific employment indicators that provide insights into the overall labor market dynamics. By categorizing workers into distinct groups based on their employment and unemployment statuses, analysts can gain deeper insights into labor market trends, identify vulnerable populations, and formulate targeted policies to address specific workforce challenges."
    }
]
def generate_card_body(groups):
    body_content = []
    for group in groups:
        body_content.append(html.H4(group['name'], className="mt-3"))
        body_content.append(html.P(group['description'], className="mb-4"))
    return body_content

intro = dbc.Row(
            dbc.Col(
                dbc.Card(
                    [
                        card("Unemployment Analysis (Recession Vs Covid)"),
                        dbc.CardBody(generate_card_body(worker_groups))
                    ]
                ),
                width=12
            )
        )

Housing Introduction

In [82]:
worker_groups = [
    {
        'name':'Introduction',
        'description':'Impacts of Great Recession (Dec 2007 to Jun 2009) and the COVID-19 pandemic in 2020 in Aspect of Housing in California, Florida, New York, and Texas.This Analysis provide insigts of three categories of unemployment Level in these states.'
    },
    {
        "name": "Housing Price Index (HPI)",
        "description": "The Housing Price Index (HPI) is a metric used to measure changes in the price of residential properties over time. It tracks the movement of prices for single-family homes, condos, and other residential properties in a given area. HPI provides valuable information about the overall health and direction of the housing market. During recessions and periods of economic turmoil, the HPI can indicate whether home values are rising or falling, providing insights into consumer confidence, investment patterns, and the broader economic landscape."
    },
    {
        "name": "Housing Permits",
        "description": "Housing permits refer to the authorization granted by local governments for the construction or renovation of residential properties. The number of housing permits issued is a leading indicator of future construction activity and serves as a barometer for the health of the housing market. During recessions, housing permits tend to decline as both consumers and developers become more cautious amid economic uncertainty. Conversely, during periods of economic expansion, an increase in housing permits signals growing confidence in the market and potential for economic growth."
    },
    {
        "name": "Home Price Tiers (Top/Bottom)",
        "description": "Home price tiers categorize residential properties into different price segments based on their value. Typically, these tiers include top-tier homes (higher-end properties) and bottom-tier homes (more affordable properties). During recessions and economic downturns, the impact on home prices can vary across these tiers. Historically, bottom-tier home prices tend to be more resilient during economic downturns compared to top-tier properties, as demand for affordable housing remains relatively stable. Conversely, top-tier home prices may experience greater volatility as high-end buyers adjust their spending habits in response to economic uncertainty."
    }
]
def generate_card_body(groups):
    body_content = []
    for group in groups:
        body_content.append(html.H4(group['name'], className="mt-3"))
        body_content.append(html.P(group['description'], className="mb-4"))
    return body_content

intro_housing = dbc.Row(
            dbc.Col(
                dbc.Card(
                    [
                        card("Housing Market Analysis (Recession Vs Covid)"),
                        dbc.CardBody(generate_card_body(worker_groups))
                    ]
                ),
                width=12
            )
        )

INtro Economic wellbeing

In [83]:
worker_groups = [
    {
        'name':'Introduction',
        'description':'Impacts of Great Recession (Dec 2007 to Jun 2009) and the COVID-19 pandemic in 2020 in Aspect of Economic well being in California, Florida, New York, and Texas.This Analysis provide insigts of three categories of unemployment Level in these states.'
    },
    {
        "name": "Total Personal Income",
        "description": "Total personal income encompasses all sources of income received by individuals within a specified geographic area or population. This includes wages, salaries, investment returns, government benefits, and other forms of earnings. Tracking total personal income provides a comprehensive measure of the financial resources available to individuals and households. Increases in total personal income typically indicate economic growth, improved employment opportunities, and enhanced purchasing power, contributing to overall economic well-being."
    },
    {
        "name": "Minimum Wage",
        "description": "Minimum wage refers to the legally mandated minimum hourly wage that employers must pay to their workers. It serves as a baseline for ensuring fair compensation and protecting workers from exploitation. Monitoring minimum wage levels is crucial for assessing income inequality, labor market dynamics, and the ability of workers to meet their basic needs. Adjustments to the minimum wage can impact consumer spending, poverty rates, and overall economic stability, making it a significant factor in determining economic well-being, particularly for low-income earners."
    },
    {
        "name": "Civilian Labor Force",
        "description": "The civilian labor force comprises individuals who are either employed or actively seeking employment within the labor market. It represents the pool of available workers and plays a central role in driving economic activity and productivity. Changes in the size and composition of the civilian labor force can reflect shifts in workforce participation, unemployment rates, and labor market dynamics, all of which influence economic well-being. A growing labor force may indicate expanding opportunities and economic vitality, while a shrinking or stagnant labor force could signal challenges such as unemployment or demographic trends."
    },
    {
        "name": "Per Capita Income:",
        "description": "Per capita income measures the average income earned per person within a specific population or geographic area. It is calculated by dividing the total income generated by the population by the total number of individuals. Per capita income provides insights into income distribution and standards of living, helping to assess the overall prosperity and economic well-being of a society. Increases in per capita income suggest rising living standards, improved access to goods and services, and enhanced quality of life for residents."
    }
]
def generate_card_body(groups):
    body_content = []
    for group in groups:
        body_content.append(html.H4(group['name'], className="mt-3"))
        body_content.append(html.P(group['description'], className="mb-4"))
    return body_content

intro_economic = dbc.Row(
            dbc.Col(
                dbc.Card(
                    [
                        card("Economic Well-being Analysis (Recession Vs Covid)"),
                        dbc.CardBody(generate_card_body(worker_groups))
                    ]
                ),
                width=12
            )
        )

Unemployment tabs 

In [84]:
unemptabs = [ 
  
tab_name(intro,'Introduction(Unemployment)'), 
tab_name(drop_down_5_tab,'Unemployment Analysis'), 
tab_name(drop_down_3_tab,'Unemployment(Recession)'),
tab_name(drop_down_4_tab,'Unemployment(Covid)'),
tab_name(intro_housing,'Introduction(Housing)'),
tab_name(drop_down_6_tab,'Housing(Recession)'),
tab_name(drop_down_7_tab,'Housing(Covid)'),
tab_name(drop_down_8_tab,'Housing Analysis'),
tab_name(intro_economic,'Introduction(Economic well-being)'),
tab_name(drop_down_9_tab,'Economic Well Being Recession Analysis'),
tab_name(drop_down_10_tab,'Economic Well Being Covid Analysis'),
]

In [85]:
app.layout = dbc.Container(
    [
        dbc.Tabs(unemptabs)
    ]
)

In [86]:
if __name__ == "__main__":
    app.run_server(
        debug=True,
        external_stylesheets=[dbc.themes.LUMEN],
        port=8008,
    )

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\2896854717.py:16: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\29088592.py:16: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\2896854717.py:16: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\Karthi\AppData\Local\Temp\ipykernel_11596\29088592.py:16: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

